In [1]:
import pandas as pd
import re
from sklearn.feature_extraction.text import TfidfVectorizer
import numpy as np
from scipy.sparse import csr_matrix
import sparse_dot_topn.sparse_dot_topn as ct

pd.set_option('display.max_colwidth', -1)
df = pd.read_csv('CSV/TransfixLeadConDups.csv',  dtype=str)
print(df.shape)
df.head(2)

(79311, 2)


/opt/anaconda3/lib/python3.6/site-packages/ipykernel_launcher.py:8: FutureWarning: Passing a negative integer is deprecated in version 1.0 and will not be supported in future version. Instead, use None to not limit the column width.
  


,Id,Email
0,00Q1J00000dhNbeUAE,miistrategic@gmail.com
1,00Q1J00000gX9XmUAK,mastertrans.dip@gmail.com


In [2]:
# Clean the data
df.dropna()
# df['Email'] = df['Email'].str.replace('[^a-zA-Z]', '')
# df['Email'] = df['Email'].str.replace(r'[^\w\s]+', '')

contact_emails = df['Email']

In [3]:
def ngrams(string, n=3):
    string = re.sub(r'[,-./]|\sBD',r'', string)
    ngrams = zip(*[string[i:] for i in range(n)])
    return [''.join(ngram) for ngram in ngrams]


In [4]:
# get Tf-IDF Matrix
vectorizer = TfidfVectorizer(min_df=1, analyzer=ngrams)
tf_idf_matrix = vectorizer.fit_transform(contact_emails.apply(lambda x: np.str_(x)))


In [5]:
def awesome_cossim_top(A, B, ntop, lower_bound=0):
    # force A and B as a CSR matrix.
    # If they have already been CSR, there is no overhead
    A = A.tocsr()
    B = B.tocsr()
    M, _ = A.shape
    _, N = B.shape
 
    idx_dtype = np.int32
 
    nnz_max = M*ntop
 
    indptr = np.zeros(M+1, dtype=idx_dtype)
    indices = np.zeros(nnz_max, dtype=idx_dtype)
    data = np.zeros(nnz_max, dtype=A.dtype)

    ct.sparse_dot_topn(
        M, N, np.asarray(A.indptr, dtype=idx_dtype),
        np.asarray(A.indices, dtype=idx_dtype),
        A.data,
        np.asarray(B.indptr, dtype=idx_dtype),
        np.asarray(B.indices, dtype=idx_dtype),
        B.data,
        ntop,
        lower_bound,
        indptr, indices, data)

    return csr_matrix((data,indices,indptr),shape=(M,N))

In [6]:
matches = awesome_cossim_top(tf_idf_matrix, tf_idf_matrix.transpose(), 10, 0.7)


In [7]:
def get_matches_df(sparse_matrix, email_vector,email_ids, top=5):
    non_zeros = sparse_matrix.nonzero()
    
    sparserows = non_zeros[0]
    sparsecols = non_zeros[1]
    
    
    if top:
        nr_matches = top
    else:
        nr_matches = sparsecols.size
    left_name_Ids = np.empty([nr_matches], dtype=object)
    right_name_Ids = np.empty([nr_matches], dtype=object)
    
    left_side = np.empty([nr_matches], dtype=object)
    right_side = np.empty([nr_matches], dtype=object)
    similairity = np.zeros(nr_matches)
    
    for index in range(nr_matches):        
        left_name_Ids[index] = email_ids[sparserows[index]]
        left_side[index] = email_vector[sparserows[index]]

        right_name_Ids[index] = email_ids[sparsecols[index]]
        right_side[index] = email_vector[sparsecols[index]]
        similairity[index] = sparse_matrix.data[index]
    
    return pd.DataFrame({
                        'SFDC_ID':  left_name_Ids,
                        'left_side': left_side,
                        'right_SFDC_ID':right_name_Ids,
                          'right_side': right_side,
                           'similairity': similairity})

In [8]:
name_Ids = df['Id']
matches_df = get_matches_df(matches, contact_emails,name_Ids, top=79311)






In [9]:
matches_df = matches_df[matches_df['similairity'] > 0.9] 




In [10]:
matches_df.sort_values('SFDC_ID' )



,SFDC_ID,left_side,right_SFDC_ID,right_side,similairity
17684,0031J00001B0zPpQAJ,rculver@condadogroup.com,0031J00001B0zPpQAJ,rculver@condadogroup.com,1.0
17689,0031J00001B1GzJQAV,cs@pegasushomefashions.com,0031J00001B1GzJQAV,cs@pegasushomefashions.com,1.0
17688,0031J00001B1LpLQAV,kates@vivbev.com,0031J00001B1LpLQAV,kates@vivbev.com,1.0
17692,0031J00001B1QMEQA3,lisa.temple@tileshop.com,0031J00001B1QMEQA3,lisa.temple@tileshop.com,1.0
17740,0031J00001B35YEQAZ,dan.drieselman@searshc.com,0031J00001B35YEQAZ,dan.drieselman@searshc.com,1.0
...,...,...,...,...,...
15067,00Q1J00000gZCq3UAG,jmehn@gantalogistics.com,0031J00001kM8dYQAS,jmehn@gantalogistics.com,1.0
15069,00Q1J00000gZCq4UAG,amtransportgroupllc@gmail.com,0031J00001kMOrKQAW,amtransportgroupllc@gmail.com,1.0
15070,00Q1J00000gZCq4UAG,amtransportgroupllc@gmail.com,0031J00001kM9FNQA0,amtransportgroupllc@gmail.com,1.0
15071,00Q1J00000gZCq4UAG,amtransportgroupllc@gmail.com,00Q1J00000gZCq4UAG,amtransportgroupllc@gmail.com,1.0


In [11]:
mask = matches_df['SFDC_ID'] == matches_df['right_SFDC_ID'] 

#if mask is true item in df['SFDC_ID'] is selected else item in df['right_SFDC_ID'] is selected

matches_df[mask]



,SFDC_ID,left_side,right_SFDC_ID,right_side,similairity
0,00Q1J00000dhNbeUAE,miistrategic@gmail.com,00Q1J00000dhNbeUAE,miistrategic@gmail.com,1.0
2,00Q1J00000gX9XmUAK,mastertrans.dip@gmail.com,00Q1J00000gX9XmUAK,mastertrans.dip@gmail.com,1.0
4,00Q1J00000gXAzXUAW,eminent.retha@gmail.com,00Q1J00000gXAzXUAW,eminent.retha@gmail.com,1.0
6,00Q1J00000exKUOUA2,dispatch@caintransport.com,00Q1J00000exKUOUA2,dispatch@caintransport.com,1.0
9,00Q1J00000dh4hiUAA,milfort1@live.com,00Q1J00000dh4hiUAA,milfort1@live.com,1.0
...,...,...,...,...,...
79305,0031J00001iMpboQAC,ejamison@amleo.com,0031J00001iMpboQAC,ejamison@amleo.com,1.0
79306,0031J00001iN43jQAC,m.crawford@haydenbeverage.com,0031J00001iN43jQAC,m.crawford@haydenbeverage.com,1.0
79307,0031J00001iMSy1QAG,cecilia.cordoncillo@commscope.com,0031J00001iMSy1QAG,cecilia.cordoncillo@commscope.com,1.0
79309,0031J00001iMOvFQAW,dennis.king@dxpe.com,0031J00001iMOvFQAW,dennis.king@dxpe.com,1.0


In [12]:
# Get names of indexes for which column mask == true
indexNames = matches_df[ mask ].index
# Delete these row indexes from dataFrame
matches_df.drop(indexNames , inplace=True)



In [13]:
matches_df.sort_values('similairity',ascending=True)



,SFDC_ID,left_side,right_SFDC_ID,right_side,similairity
77702,0031J00001epVihQAE,f.metz@vilebrequin.com,0031J00001epViSQAU,p.metz@vilebrequin.com,0.901381
77445,0031J00001epViSQAU,p.metz@vilebrequin.com,0031J00001epVihQAE,f.metz@vilebrequin.com,0.901381
33912,0031J00001O5HFUQA3,cmiller@buchananhauling.com,0031J00001O5HGyQAN,jmiller@buchananhauling.com,0.901688
31521,0031J00001O5HGyQAN,jmiller@buchananhauling.com,0031J00001O5HFUQA3,cmiller@buchananhauling.com,0.901688
59707,0031J00001enwXRQAY,risack@thecheesecakefactory.com,0031J00001Orn8kQAB,lschack@thecheesecakefactory.com,0.901964
...,...,...,...,...,...
14230,00Q1J00000gXK2ZUAW,blippitt@qfstransportation.com,0031J00001iMsINQA0,blippitt@qfstransportation.com,1.000000
8605,00Q1J00000exFJTUA2,eddie.williams@specialtymfginc.com,0031J00001emg0dQAA,eddie.williams@specialtymfginc.com,1.000000
10839,00Q1J00000exFY3UAM,diversitygo@gmail.com,0031J00001bycy5QAA,diversitygo@gmail.com,1.000000
7265,00Q1J00000ewUY3UAM,cruglogisticsllc@gmail.com,0031J00001cvGmOQAU,cruglogisticsllc@gmail.com,1.000000


In [14]:
matches_df.to_csv ('./Results/LeadContactDups.csv',index = False, header=True)

In [15]:
mask = matches_df['SFDC_ID'] < matches_df['right_SFDC_ID'] 

# creates a new column checking True vs False, 

#if mask is true item in df['SFDC_ID'] is selected else item in df['right_SFDC_ID'] is selected

matches_df['col1'] = matches_df['SFDC_ID'].where(mask, matches_df['right_SFDC_ID'])

#same as above but a column for df['right_SFDC_ID']
matches_df['col2'] = matches_df['right_SFDC_ID'].where(mask, matches_df['SFDC_ID'])

# checks for duplicates in `col1` and `col2` and removes last duplicate
matches_df = matches_df.drop_duplicates(subset=['col1'])



In [16]:
matches_df.sort_values('similairity',ascending=True)

matches_df['similairity']= matches_df['similairity'].astype(str)


In [17]:
matches_df.to_csv ('./Results/LeadContactDups2.csv',index = False, header=True)



In [18]:
test_df = matches_df.groupby(['SFDC_ID', 'left_side' ], as_index=False)[['right_SFDC_ID', 'right_side', 'similairity']].agg(lambda x: ','.join(x))

test_df.head(500)


,SFDC_ID,left_side,right_SFDC_ID,right_side,similairity
0,0031J00001O5HGyQAN,jmiller@buchananhauling.com,"0031J00001O5Hv0QAF,0031J00001O5HFUQA3","dmiller@buchananhauling.com,cmiller@buchananhauling.com","0.9150357300953935,0.9016883755884708"
1,0031J00001OimvwQAB,mikerobinson@carlislefsp.com,003o000000wObweAAC,mikerobinson@carlislefsp.com,1.0000000000000002
2,0031J00001Orn8kQAB,lschack@thecheesecakefactory.com,0031J00001enwXRQAY,risack@thecheesecakefactory.com,0.9019635755563277
3,0031J00001P48IHQAZ,dkaduke@limitedbrands.com,0031J00001P48MnQAJ,dkaduke@limitedbrands.com,0.9999999999999998
4,0031J00001P48McQAJ,robert.mcrae@pepsico.com,0031J00001T50Z3QAJ,robert.mcrae@pepsico.com,1.0
...,...,...,...,...,...
495,00Q1J00000ewM8MUAU,donald@sdmtrans.com,0031J00001cv4hnQAA,donald@sdmtrans.com,0.9999999999999999
496,00Q1J00000ewM8NUAU,glongtin1@yahoo.com,0031J00001cv4hoQAA,glongtin1@yahoo.com,0.9999999999999994
497,00Q1J00000ewM8OUAU,samratransportllc@gmail.com,0031J00001cv4hpQAA,samratransportllc@gmail.com,1.0000000000000002
498,00Q1J00000ewM8PUAU,emilioromero1276@gmail.com,0031J00001cv4i6QAA,emilioromero1276@gmail.com,1.0


In [19]:
test_df.to_csv ('./Results/LeadContactContactDups3.csv',index = False, header=True)

